In [17]:
import re
import pandas as pd
from functools import lru_cache
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm

In [ ]:
nltk.download("punkt_tab")
nltk.download("stopwords")

Тестовый набор данных содержит информацию о проектах, поданных на конкурс. Первый столбец - номер, где первые две цифры обозначают год, а следующая цифра — номер конкурса. Например, номер 22-2-006088 означает 2022 год, 2 конкурс. Эта информация будет полезна при поиске похожих текстов.
Второй столбец содержит текст с кратким описанием проекта и обоснованием его социальной значимости.

In [ ]:
df = pd.read_csv("applications_dataset.csv")
df

In [4]:
morph = pymorphy3.MorphAnalyzer()


@lru_cache(maxsize=100000)
def get_normal_form(word):
    return morph.parse(word.lower())[0].normal_form


def clean_text(text):
    """
    Функция для очистки текста.
    """
    # Удаление URL
    text = re.sub(r"http\S+", "", text)
    # Удаление спецсимволов и чисел
    text = re.sub(r"[\n%\t•,\"'\[\]{}()«»1234567890№:;!]", "", text)
    # Заменяем дефисы на пробел
    text = re.sub(r"[-–]", " ", text)
    # Заменяем точки на пробел
    text = re.sub(r"\.", " ", text)
    # Удаление лишних пробелов
    text = re.sub(
        r"\s+", " ", text
    ).strip()  # Удаляем лишние пробелы и обрезаем пробелы по краям
    return text


def lemmatized_text(text):
    """
    Функция лемматизирует исходный текст.
    """
    words = word_tokenize(text)
    # Лемматизация слов с использованием кэша
    lemmatized_words = [get_normal_form(word) for word in words]
    lemmatized_text = " ".join(lemmatized_words)
    return lemmatized_text


def del_stop_words(text):
    """
    Функция для удаления стоп-слов.
    """
    stop_words = stopwords.words("russian")
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = " ".join(filtered_words)
    return filtered_text

In [5]:
# Создание шинглов по словам
def create_shingles(tokens, k=3):
    """
    Функция создает шинглы.
    """
    tokens = tokens.split()
    shingl = [" ".join(tokens[i : i + k]) for i in range(len(tokens) - k + 1)]
    return shingl


def create_minhash(tokens, num_perm=128, k=3):
    """
    Функция создает MinHash.
    """
    m = MinHash(num_perm=num_perm, seed=1631997)
    for shingle in tokens:
        m.update(shingle.encode("utf8"))
    return m

In [6]:
def get_shingles_position(k, document):
    """
    Находит положение шинглов в тексте.
    """
    shingles_positions = {}
    words = document.split()

    for i in range(0, len(words) - k + 1):
        shingle_words = words[i : i + k]
        shingle = " ".join(shingle_words)
        if shingle in shingles_positions:
            shingles_positions[shingle].append((i, i + k - 1))
        else:
            shingles_positions[shingle] = [(i, i + k - 1)]

    return shingles_positions


def merge_positions(positions):
    """
    Объединяет позиции шинглов, если они перекрываются или соприкасаются.
    """
    if not positions:
        return []

    # Собираем все позиции в один список
    shingle_positions = []
    for el in positions.values():
        shingle_positions.extend(el)

    # Сортируем позиции по началу
    shingle_positions.sort()

    # Объединяем позиции
    merged = []
    current_start, current_end = shingle_positions[0]

    for start, end in shingle_positions[1:]:
        if start <= current_end + 1:  # Если позиции перекрываются или соприкасаются
            current_end = max(current_end, end)
        else:
            merged.append([current_start, current_end])  # Добавляем текущий интервал
            current_start, current_end = start, end

    # Добавляем последний интервал
    merged.append([current_start, current_end])
    return merged


def search_shingles_position(ordinalnumber1, ordinalnumber2):
    """
    Находит позиции совпадающих шинглов в двух текстах.
    """
    # Получаем тексты
    text_original = df[df["ordinalnumber"] == ordinalnumber1]["lemmatized_text"].values[
        0
    ]
    text_suspected = df[df["ordinalnumber"] == ordinalnumber2][
        "lemmatized_text"
    ].values[0]

    # Получаем позиции шинглов
    shingles_positions_original = get_shingles_position(3, text_original)
    shingles_positions_suspected = get_shingles_position(3, text_suspected)

    # Находим совпадающие шинглы
    plagiarized_shingles = {}
    for shingle, positions in shingles_positions_suspected.items():
        if shingle in shingles_positions_original:
            plagiarized_shingles[shingle] = positions

    # Объединяем позиции
    merged = merge_positions(plagiarized_shingles)

    return merged

In [7]:
def calculate_lsh_similarity(df, threshold=0.1):
    """
    Cчитает LSH и находит потенциальные пары заявок, в которых есть заимствования.
    """
    print("Start calculate Lsh..")

    # Создаем LSH объект
    lsh = MinHashLSH(threshold=threshold, num_perm=128)

    # Добавляем заявки и их minhash значения в LSH объект
    for ordinalnumber, minhash in zip(df["ordinalnumber"], df["minhash"]):
        lsh.insert(ordinalnumber, minhash)

    print("LSH Similarity computed")

    results = []

    for original_ordinalnumber in tqdm(
        df["ordinalnumber"], desc="Processing LSH Queries"
    ):
        # Ищем похожие заявки с помощью LSH
        similar_items = lsh.query(
            df[df["ordinalnumber"] == original_ordinalnumber]["minhash"].values[0]
        )
        for similar_ordinalnumber in similar_items:
            # Проверка, что оригинальный текст был добавлен раньше по времени, чем похожий текст
            if original_ordinalnumber < similar_ordinalnumber:
                results.append((original_ordinalnumber, similar_ordinalnumber))

    return results

In [8]:
def final_search_for_similar_requests(results, trash=40):
    """
    Находит номера похожих заявок со сходством больше, чем trash и возвращает словарь похожих пар заявок и их процент сходства.
    """

    results_application = []

    for ordinalnumber1, ordinalnumber2 in tqdm(
        results, desc="Processing Shingles Comparison"
    ):
        # Получаем значения шинглов
        shingles1 = set(df[df["ordinalnumber"] == ordinalnumber1]["shingles"].values[0])
        shingles2 = set(df[df["ordinalnumber"] == ordinalnumber2]["shingles"].values[0])

        # Находим пересечение шинглов
        common_shingles = shingles1.intersection(shingles2)

        # Рассчитываем процент совпадения по коэффициенту Сёренсена
        percent_match = (
            (2 * len(common_shingles)) / (len(shingles1) + len(shingles2)) * 100
        )
        if percent_match >= trash:
            shingles_pos = search_shingles_position(ordinalnumber1, ordinalnumber2)
            results_application.append(
                {
                    "applications": (ordinalnumber1, ordinalnumber2),
                    "similarity": percent_match,
                    "shingles_positions": shingles_pos,
                }
            )

    return results_application

In [ ]:
# Очищаем и лемматизируем текст
df["clean_text"] = df["text"].apply(clean_text)
df["lemmatized_text"] = df["clean_text"].apply(lemmatized_text)
# Удаляем из текста стоп-слова
df["processed_text"] = df["lemmatized_text"].apply(del_stop_words)
df["processed_text"].head()

In [ ]:
# Разбиваем текст на шинглы
df["shingles"] = df["processed_text"].apply(create_shingles, k=3)
df["shingles"].head()

In [ ]:
# Создаем объект MinHash
df["minhash"] = df["shingles"].apply(create_minhash, k=3)
df["minhash"].head()

In [ ]:
# Подсчитыаем LSH и находим потенциальные пары заявок
results_lsh = calculate_lsh_similarity(df)
results_lsh

In [ ]:
# Находим похожие тексты, для которых сходство больше, чем trash 40%
results_application = final_search_for_similar_requests(results_lsh, trash=40)
results_application

In [ ]:
# Выводим текст похожих новостей и информацию о заимствованиях
for info in results_application:
    ordinalnumber1, ordinalnumber2 = info["applications"]
    text1 = df[df["ordinalnumber"] == ordinalnumber1]["text"].values[0]
    text2 = df[df["ordinalnumber"] == ordinalnumber2]["text"].values[0]
    print(f"Номер {ordinalnumber1}, оригинальный текст: {text1}")
    print(f"Номер {ordinalnumber2}, текст с заимстованиями: {text2}")
    print(f"Процент сходства: {info['similarity']}")
    print(f"Совпадающие позиции слов: {info['shingles_positions']}")
    print("-" * 80)